In [ ]:
!conda --version 

In [ ]:
!conda update -n base -c conda-forge conda

In [ ]:
!pip install "openbb" --no-cache-dir              

In [3]:
from openbb_terminal.sdk import openbb
# Show trending coins
openbb.crypto.disc.trending()

,Symbol,Name,market_cap Cap Rank
0,bitcoin,Bitcoin,1
1,bonk,Bonk,942
2,apecoin,ApeCoin,80
3,trust-wallet-token,Trust Wallet,97
4,unibot,Unibot,257
5,arbitrum,Arbitrum,43
6,solana,Solana,10
